In [2]:
import os
import json
import time
import random
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import pickle
import torchvision
from tqdm.notebook import tqdm

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="6"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
config = dict(
    saved_path="saved_models/resnet18.pt",
    lr=0.001, 
    EPOCHS = 5,
    BATCH_SIZE = 8,
    IMAGE_SIZE = 128,
    TRAIN_VALID_SPLIT = 0.2,
    device=device,
    SEED = 42,
    pin_memory=True,
    num_workers=2,
    USE_AMP = True,
    channels_last=False)

random.seed(config['SEED'])
np.random.seed(config['SEED'])
torch.manual_seed(config['SEED'])
torch.cuda.manual_seed(config['SEED'])

In [6]:
torch.backends.cudnn.benchmarks = True
torch.backends.cudnn.deterministic = True

torch.backends.cuda.matmul.allow_tf32 = True

# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((config['IMAGE_SIZE'],config['IMAGE_SIZE'])),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAutocontrast(0.5),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((config['IMAGE_SIZE'],config['IMAGE_SIZE'])),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((config['IMAGE_SIZE'],config['IMAGE_SIZE'])),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Triplet Loss Model

In [8]:
a_file = open("dataset.pkl", "rb")
dataset = pickle.load(a_file)
print(len(set(dataset['labels'][:500])))

1


In [10]:
class Custom_data(Dataset):
    def __init__(self, dataset, transform = data_transforms, train=True):
        super(Custom_data,self).__init__()
        self.train_transforms = transform['test']
        self.test_transforms = transform['test']
        self.is_train = train
        self.to_pil = transforms.ToPILImage()
        
        if self.is_train:
            self.images = dataset['images']
            self.labels = np.array(dataset['labels'])
            self.index = np.array(list(range(len(self.labels))))
        
        else:
            self.images = dataset['images']
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        anchor_img = self.images[item]
        
        if self.is_train:
            anchor_label = self.labels[item]
            positive_list = self.index[self.index!=item][self.labels[self.index!=item]==anchor_label]

            positive_item = random.choice(positive_list)
            positive_img = self.images[positive_item]

            negative_list = self.index[self.index!=item][self.labels[self.index!=item]!=anchor_label]
            negative_item = random.choice(negative_list)
            negative_img = self.images[negative_item]

            if self.train_transforms:
                anchor_img = self.train_transforms(self.to_pil(anchor_img))
                positive_img = self.train_transforms(self.to_pil(positive_img))
                negative_img = self.train_transforms(self.to_pil(negative_img))

                return anchor_img, positive_img, negative_img, anchor_label
        
        else:
            if self.transform:
                anchor_img = self.test_transforms(self.to_pil(anchor_img))
            return anchor_img

In [11]:
train_ds = Custom_data(dataset, train=True)
train_loader = DataLoader(train_ds, batch_size=config['BATCH_SIZE'], shuffle=True,pin_memory = config['pin_memory'])
a = iter(train_loader)
b = next(a)
print(b[0].shape, b[1].shape, b[2].shape, b[3].shape)
print(len(train_loader))

torch.Size([8, 3, 128, 128]) torch.Size([8, 3, 128, 128]) torch.Size([8, 3, 128, 128]) torch.Size([8])
12500


In [12]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

# Reloading saved weights of the model.
densenet = models.densenet121(pretrained = True)
densenet.classifier = Identity()
model = densenet
model = model.to(config['device'])
model.load_state_dict(torch.load('saved_models/densenet121_triplet.pt'))

<All keys matched successfully>

In [13]:
len(train_loader)

12500

In [14]:
# storing the feature embedding vector in train_results list
train_results = []
labels = []
count = 0
model.eval()
with torch.no_grad():
    for img, _, _, label in train_loader:
        if count<600: # considering 600 batches
            train_results.append(model(img.to(device)).cpu().numpy())
            labels.append(label)
            count+=1
        else:
            break
        
train_results = np.concatenate(train_results)
labels = np.concatenate(labels)
print(train_results.shape, labels.shape)

(4800, 1024) (4800,)


In [27]:
# Train test split
l = (len(labels)*8)//10
x_train,y_train = train_results[:l],labels[:l]
x_test,y_test = train_results[l:],labels[l:]

In [16]:
# Catboost classifier
# !pip install catboost

from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=100)
cat.fit(x_train,y_train)

Learning rate set to 0.5
0:	learn: 5.1245961	total: 3.32s	remaining: 5m 28s
1:	learn: 4.9425648	total: 6.6s	remaining: 5m 23s
2:	learn: 4.7737838	total: 9.79s	remaining: 5m 16s
3:	learn: 4.6494719	total: 13.9s	remaining: 5m 33s
4:	learn: 4.5194368	total: 17.8s	remaining: 5m 38s
5:	learn: 4.3885183	total: 21.4s	remaining: 5m 36s
6:	learn: 4.2911668	total: 24.8s	remaining: 5m 30s
7:	learn: 4.1930719	total: 28s	remaining: 5m 22s
8:	learn: 4.0866090	total: 31.7s	remaining: 5m 20s
9:	learn: 3.9937066	total: 34.9s	remaining: 5m 14s
10:	learn: 3.9055630	total: 38.4s	remaining: 5m 10s
11:	learn: 3.8164290	total: 41.7s	remaining: 5m 5s
12:	learn: 3.7296395	total: 45.1s	remaining: 5m 1s
13:	learn: 3.6790027	total: 48.8s	remaining: 4m 59s
14:	learn: 3.5881671	total: 52.7s	remaining: 4m 58s
15:	learn: 3.4957695	total: 56.2s	remaining: 4m 55s
16:	learn: 3.4166463	total: 59.6s	remaining: 4m 50s
17:	learn: 3.3553706	total: 1m 2s	remaining: 4m 45s
18:	learn: 3.2985342	total: 1m 5s	remaining: 4m 40s
19

In [29]:
y_pred = cat.predict(x_test)
print('accuracy',metrics.accuracy_score(y_test,y_pred))
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
                                                          
print("precision_score: ",precision)
print("recall_score: ",recall)
print("f1_score: ",f1)

accuracy 0.408125
precision_score:  0.4783128081847848
recall_score:  0.408125
f1_score:  0.40659339871759487


# Center Loss Model

In [16]:
my_path = '../dataset/miniimgnet_dlassignment/tinyimgnet/tiny-imagenet-200/train/'
images = torchvision.datasets.ImageFolder(root=my_path,transform=data_transforms['train'])
print(len(images))
train_data,valid_data = torch.utils.data.dataset.random_split(images,[90000,10000])


train_dl = torch.utils.data.DataLoader(dataset=train_data,batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])
valid_dl = torch.utils.data.DataLoader(dataset = valid_data,batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])

100000


In [17]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes=200, feat_dim=200, use_gpu=True):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.use_gpu = use_gpu

        if self.use_gpu:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
        else:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, labels):
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long()
        if self.use_gpu: classes = classes.cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss

In [18]:
def evaluation(model,test_dl,test_data):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    preds = []
    pred_labels = []
    labels = []

    with torch.no_grad():
                for x,y in test_dl:
                    x = x.to(config['device'])
                    y = y.to(config['device']) #CHW --> #HWC
                    valid_logits = model(x)
                    predict_prob = F.softmax(valid_logits)
                    _,predictions = predict_prob.max(1)
                    predictions = predictions.to('cpu')

                    _, valid_preds = torch.max(valid_logits, 1)
                    valid_loss = criterion(valid_logits,y)
                    running_loss += valid_loss.item() * x.size(0)
                    running_corrects += torch.sum(valid_preds == y.data)
                    total += y.size(0)
                    predict_prob = predict_prob.to('cpu')

                    pred_labels.extend(list(predictions.numpy()))
                    preds.extend(list(predict_prob.numpy()))
                    y = y.to('cpu')
                    labels.extend(list(y.numpy()))

    epoch_loss = running_loss / len(test_data)
    epoch_acc = running_corrects.double() / len(test_data)
    #print("Test Loss is {}".format(epoch_loss))
    print("Test Accuracy is {}".format(epoch_acc.cpu()))
    return np.array(labels),np.array(pred_labels),np.array(preds)

In [20]:
densenet = models.densenet121(pretrained = True)
densenet.classifier = nn.Linear(in_features = 1024, out_features = 200, bias = True)
model = densenet
model = model.to(config['device'])
model.load_state_dict(torch.load('saved_models/densenet121_center.pt'))

criterion = nn.CrossEntropyLoss()

In [21]:
labels, pred_labels, preds = evaluation(model,valid_dl,valid_data)

Test Accuracy is 0.3503


In [22]:
precision, recall, f1, _ = precision_recall_fscore_support(labels, pred_labels, average='weighted')

print("precision_score: ",precision)
print("recall_score: ",recall)
print("f1_score: ",f1)

precision_score:  0.3196631341429935
recall_score:  0.3503
f1_score:  0.31600115823036806


# Cross Entropy loss model

In [45]:
my_path = '../dataset/miniimgnet_dlassignment/tinyimgnet/tiny-imagenet-200/train/'
images = torchvision.datasets.ImageFolder(root=my_path,transform=data_transforms['train'])
print(len(images))
train_data,valid_data = torch.utils.data.dataset.random_split(images,[90000,10000])


100000


In [46]:
train_dl = torch.utils.data.DataLoader(dataset=train_data,batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])
valid_dl = torch.utils.data.DataLoader(dataset = valid_data,batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])

In [47]:
def evaluation(model,test_dl,test_data):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    preds = []
    pred_labels = []
    labels = []

            # Disable gradient calculation for validation or inference using torch.no_rad()
    with torch.no_grad():
                for x,y in test_dl:
                    x = x.to(config['device'])
                    y = y.to(config['device']) #CHW --> #HWC
                    valid_logits = model(x)
                    predict_prob = F.softmax(valid_logits)
                    _,predictions = predict_prob.max(1)
                    predictions = predictions.to('cpu')

                    _, valid_preds = torch.max(valid_logits, 1)
                    valid_loss = criterion(valid_logits,y)
                    running_loss += valid_loss.item() * x.size(0)
                    running_corrects += torch.sum(valid_preds == y.data)
                    total += y.size(0)
                    predict_prob = predict_prob.to('cpu')

                    pred_labels.extend(list(predictions.numpy()))
                    preds.extend(list(predict_prob.numpy()))
                    y = y.to('cpu')
                    labels.extend(list(y.numpy()))

    epoch_loss = running_loss / len(test_data)
    epoch_acc = running_corrects.double() / len(test_data)
    print("Test Loss is {}".format(epoch_loss))
    print("Test Accuracy is {}".format(epoch_acc.cpu()))
    return np.array(labels),np.array(pred_labels),np.array(preds)

In [48]:
densenet = models.densenet121(pretrained = True)
densenet.classifier = nn.Linear(in_features = 1024, out_features = 200, bias = True)
model = densenet
model = model.to(config['device'])
model.load_state_dict(torch.load('saved_models/densenet121_crossentropy.pt'))

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [49]:
labels, pred_labels, preds = evaluation(model,valid_dl,valid_data)

Test Loss is 1.9161838850021362
Test Accuracy is 0.5405


In [50]:
precision, recall, f1, _ = precision_recall_fscore_support(labels, pred_labels, average='weighted')

print("precision_score: ",precision)
print("recall_score: ",recall)
print("f1_score: ",f1)

precision_score:  0.5581511134618601
recall_score:  0.5405
f1_score:  0.5340788106109255
